# WEEK 3 CODE

### Content:
* TASK 1
* TASK 2
* TASK 3
* TASK 4
* TASK 5


In [3]:
#IMPORTS
import sys
import cv2
import os
import os.path
import numpy as np
import math
np.set_printoptions(threshold=np.inf)
import matplotlib.patches as patches
import ml_metrics as metrics
import pickle
from matplotlib import pyplot as plt
from os.path import isfile, join
from os import listdir
from sklearn import preprocessing
from sklearn.metrics import recall_score,precision_score,f1_score
from scipy.spatial import distance
from scipy.signal import convolve2d
from scipy.fftpack import dct
from scipy import ndimage
from skimage.feature import local_binary_pattern
from skimage.feature import hog
from skimage.filters import roberts, sobel, sobel_h, sobel_v
from skimage.morphology import skeletonize, dilation, square, opening, closing, erosion
from pathlib import Path
from cv2 import boundingRect
from PIL import Image, ImageDraw
import pytesseract

# TASK 1: Filter noise with linear or non-linear filters

In [2]:
# DEFINE FUNCTIONS THAT WILL BE USED IN TASK 1

def detectnoise(img):
    # Calcula la sigma del soroll. Llavors les imatges que tenen soroll tenen una sigma més gran 
    #que la resta de les imatges
    thr = 8.0
    H, W = gray.shape

    M = [[1, -2, 1],
        [-2, 4, -2],
        [1, -2, 1]]
    
    sigma = np.sum(np.sum(np.absolute(convolve2d(gray, M))))
    sigma = sigma * math.sqrt(0.5 * math.pi) / (6 * (W-2) * (H-2))
    if sigma>thr:
        return True
    else:
        return False


In [6]:
query2folder = './qst1_w3/'
#query2folder = './qsd2_w3/'
# CREEM UN DIRECTORI ON GUARDEM LES IMATGES SENSE SOROLL
path = 'qst1_w3_denoise'
if not os.path.exists(path):
    os.mkdir(path)


for filename in sorted(listdir(query2folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        #print ("\r Processing image...   {}".format(query2folder+filename), end="")
        query_img = cv2.imread(query2folder + filename)
        gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
        noise = detectnoise(gray)
        if noise:
            denoise_img = cv2.bilateralFilter(query_img, 9, 120, 120)
            denoise_img_2 = cv2.fastNlMeansDenoisingColored(denoise_img,None,15,15,10,21) 
            #plt.figure()
            #plt.imshow(denoise_img)
            #plt.figure()
            #plt.imshow(denoise_img_2)
        else:
            denoise_img_2 = query_img
        
        cv2.imwrite(os.path.join(path,filename), denoise_img_2)

            
        

# TASK 2: Implement text descriptors

In [6]:
def find_part_box(crop,ratio,box,a,counter,center):
    pos_bboxes = []
    pos_bbox = []
    area = []

    for c, color in enumerate(center):
        pos = np.where(crop == color)
        aux = np.zeros(crop.shape[:2], dtype='uint8')
        aux[pos[0], pos[1]] = 1
        aux = cv2.morphologyEx(aux, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
        ret, labels = cv2.connectedComponents(aux)

        for l in range(ret):
            lab_pos = np.where(labels == l)

            if (any(lab_pos[1] == 200) and any(lab_pos[1] == 250) and any(lab_pos[1] == 150)) and (
                    any(lab_pos[0] == 20) or any(lab_pos[0] == 40)
                    or any(lab_pos[0] == 60) or any(lab_pos[0] == 75)):
                continue
            else:
                aux[lab_pos[0], lab_pos[1]] = 0

        area.append(np.sum(aux))
        pos_bboxes.append(np.where(aux > 0))

    idx = sorted(range(len(area)), key=lambda k: area[k])

    if len(idx) > 1:

        aux1 = np.zeros(crop.shape[:2], dtype='uint8')
        aux1[pos_bboxes[idx[-1]][0], pos_bboxes[idx[-1]][1]] = 255
        (x, y, w, h) = cv2.boundingRect(aux1)
        aux_col1 = np.stack((aux1, aux1, aux1), axis=2)
        if (w * h) is 0:
            ratio[counter] = 0
            box[:, :, counter] = aux1
            a[counter] = area[idx[-1]]
        else:
            cv2.rectangle(aux_col1, (x, y), (x + w, y + h), (0, 0, 255), 2)
            ratio1 = (np.sum(aux1[y:y + h, x:x + w]) / 255) / (w * h)

            if not area[idx[-2]] == 0:

                aux2 = np.zeros(crop.shape[:2], dtype='uint8')
                aux2[pos_bboxes[idx[-2]][0], pos_bboxes[idx[-2]][1]] = 255
                (x, y, w, h) = cv2.boundingRect(aux2)
                aux_col2 = np.stack((aux2, aux2, aux2), axis=2)
                cv2.rectangle(aux_col2, (x, y), (x + w, y + h), (0, 0, 255), 2)

                ratio2 = (np.sum(aux2[y:y + h, x:x + w]) / 255) / (w * h)

                if ratio1 > ratio2:
                    ratio[counter] = ratio1
                    box[:, :, counter] = aux1
                    a[counter] = area[idx[-1]]
                else:
                    ratio[counter] = ratio2
                    box[:, :, counter] = aux2
                    a[counter] = area[idx[-2]]

            else:
                ratio[counter] = ratio1
                box[:, :, counter] = aux1
                a[counter] = area[idx[-1]]
                        
    return ratio,box,a

def detect_corners(mask):
    """
    Finds four points corresponding to rectangle corners

    :param mask: (ndarray) binary image
    :return: (int) points from corners
    """

    width = mask.shape[1]
    height = mask.shape[0]
    coords = np.argwhere(np.ones([height, width]))
    coords_x = coords[:, 1]
    coords_y = coords[:, 0]

    coords_x_filtered = np.extract(mask, coords_x)
    coords_y_filtered = np.extract(mask, coords_y)
    max_br = np.argmax(coords_x_filtered + coords_y_filtered)
    max_tr = np.argmax(coords_x_filtered - coords_y_filtered)
    max_tl = np.argmax(-coords_x_filtered - coords_y_filtered)
    max_bl = np.argmax(-coords_x_filtered + coords_y_filtered)

    tl_x, tl_y = int(coords_x_filtered[max_tl]), int(coords_y_filtered[max_tl])
    tr_x, tr_y = int(coords_x_filtered[max_tr]), int(coords_y_filtered[max_tr])
    bl_x, bl_y = int(coords_x_filtered[max_bl]), int(coords_y_filtered[max_bl])
    br_x, br_y = int(coords_x_filtered[max_br]), int(coords_y_filtered[max_br])

    return tl_x, tl_y, bl_x, bl_y, br_x, br_y, tr_x, tr_y

def find_box(query_img):
    
    query_img = cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB)

    resize_size = (500, 500)
    image_shape = query_img.shape[:2]
    img = cv2.resize(query_img, resize_size)

    image_dark = cv2.morphologyEx(img, cv2.MORPH_OPEN, np.ones((6, 10), np.uint8))
    image_white = cv2.morphologyEx(img, cv2.MORPH_CLOSE, np.ones((6, 10), np.uint8))

    images = np.stack((image_dark, image_white))

    ratio_top = np.zeros(2)
    box_top = np.zeros((100, 400, 2), np.uint8)
    area_top = np.zeros(2)

    ratio_bottom = np.zeros(2)
    box_bottom = np.zeros((100, 400, 2), np.uint8)
    area_bottom = np.zeros(2)

    count = 0
    for image in images:
        # plt.figure()
        # plt.imshow(image)

        Z = image.reshape((-1, 3))
        # convert to np.float32
        Z = np.float32(Z)

        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        ret, label, center = cv2.kmeans(Z, 12, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

        center = np.uint8(center)
        res = center[label.flatten()]
        res2 = res.reshape((image.shape))

        crop_top = res2[:100, 50:-50]
        crop_bottom = res2[-100:, 50:-50]

        '''plt.figure()
        plt.subplot(121)
        plt.imshow(crop_top)
        plt.subplot(122)
        plt.imshow(crop_bottom)'''

        mask = np.zeros(resize_size)

        ratio_top,box_top,area_top = find_part_box(crop_top,ratio_top,box_top,area_top,count,center)
        ratio_bottom,box_bottom,area_bottom = find_part_box(crop_bottom,ratio_bottom,box_bottom,area_bottom,count,center)

        count += 1

    if ratio_top[0] > ratio_top[1]:
        final_ratio_top = ratio_top[0]
        final_box_top = box_top[:,:,0]
    else:
        final_ratio_top = ratio_top[1]
        final_box_top = box_top[:,:,1]

    if ratio_bottom[0] > ratio_bottom[1]:
        final_ratio_bottom = ratio_bottom[0]
        final_box_bottom = box_bottom[:,:,0]
    else:
        final_ratio_bottom = ratio_bottom[1]
        final_box_bottom = box_bottom[:,:,1]

    top = np.where(final_box_top>0)
    bottom = np.where(final_box_bottom>0)

    mask = np.zeros(resize_size,np.uint8)
    mask[top[0],top[1]+50] = 1
    mask[bottom[0]+400,bottom[1]+50] = 1

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))        
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))

    mask = cv2.resize(mask,(image_shape[1],image_shape[0]))

    ret, labels = cv2.connectedComponents(mask)

    max_ratio=0

    for label in range(ret):

        if np.sum(mask[labels==label])==0:
            continue

        aux = np.zeros(image_shape,np.uint8)
        aux[labels==label]=1
        (x, y, w, h) = cv2.boundingRect(aux)

        ratio = (np.sum(aux[y:y + h, x:x + w])) / (w * h)

        if ratio >= max_ratio:
            max_ratio = ratio
            final_mask = aux

            corners = detect_corners(aux)

            bbox = [corners[0],corners[1],corners[4],corners[5]]

    return bbox

In [ ]:
query2folder = './qsd1_w3_denoise/'

bboxes=[]

for filename in sorted(listdir(query2folder)):
    if (filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(query2folder+filename), end="")
        query_img = cv2.imread(query2folder + filename)
        bbox = find_box(query_img)
        bboxes.append([bbox])
        
pickle_out = open("qst1_w3_text.pkl", "wb")
pickle.dump(bboxes, pickle_out)
pickle_out.close()

In [4]:
# Read txt from bbdd_text 
import codecs
textfolder = './bbdd_text/'
bbdd_text = []
for filename in sorted(listdir(textfolder)):
    if(filename != '.DS_Store' and ('txt' in filename)):
        with codecs.open(textfolder + filename,'r',encoding='latin1') as f:
            if  os.stat(textfolder + filename).st_size == 0:
                line = ''
                bbdd_text.append(line)
                #print(filename + line)
            else:
                for line in f.readlines():
                    lim1 = line.find("'",1)
                    lim2 = line.find("'",2)
                    bbdd_text.append(line[lim1+1:lim2])

In [5]:
# Read txt from GT 
textfolder = './qst1_w3/'
GTtext = []
for filename in sorted(listdir(textfolder)):
    if(filename != '.DS_Store' and ('txt' in filename)):
        with codecs.open(textfolder + filename,'r',encoding='latin1') as f:
            if  os.stat(textfolder + filename).st_size == 0:
                line = ''
                GTtext.append(line)
                #print(filename + line)
            else:
                for line in f.readlines():
                    lim1 = line.find("'",1)
                    lim2 = line.find("'",2) 
                    GTtext.append(line[lim1+1:lim2])

In [9]:
#DETECT TEXTS INTO BBOXES

#brew install tesseract
import pytesseract
#import pytesseract
with open('qst1_w3_text.pkl', 'rb') as fd:
    ll = pickle.load(fd)
    #print(ll)
            
query2folder = './qst1_w3_denoise/'
count = 0
query_text = []
for filename in sorted(listdir(query2folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        query_img = cv2.imread(query2folder + filename)
        img = query_img[ll[count][0][1]:ll[count][0][3],ll[count][0][0]:ll[count][0][2]]
        extractedInformation = pytesseract.image_to_string(img)
        query_text.append(extractedInformation)
        count = count + 1

In [29]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])

def computeMatrixRetrievalText(bbdd_text,query_text,K=10):
    dst = np.zeros((30, 280))
    matrix_retrieval_text = np.zeros((30,K))
    
    for query_text_index in range(0,len(query_text)):
        for bbdd_text_index in range(0,len(bbdd_text)):
            dst[query_text_index,bbdd_text_index] = levenshtein(query_text[query_text_index], bbdd_text[bbdd_text_index])
            #print(bbdd_text_index)
        matrix_retrieval_text[query_text_index,:] = np.argsort(dst[query_text_index,:])[:K]
        
     #Convertir idx a list of lists
    matrix_retrieval_text = matrix_retrieval_text.astype(int)
    matrix_retrieval_text_lst = matrix_retrieval_text.tolist()
    return matrix_retrieval_text_lst  


def evaluationTask1(matrix_retrieval, K, query_folder):  
    #LLEGIR GT
    with open(query_folder + 'gt_corresps.pkl', 'rb') as fd:
            ll = pickle.load(fd)
            #print(ll)
    gt = np.empty((0,0))

    ll_prp = np.zeros((len(ll),1))

    for i in range(0,len(ll)):
        ll_prp[i] = ll[i][0]

    ll_prp_lst = ll_prp.tolist()
    #print(ll_prp_lst)

    mapak = metrics.average_precision.mapk(ll_prp_lst, matrix_retrieval, k=K)    
    score = mapak*100
    #print("SCORE: ",score, "%")

    #GUARDAR PICKLE RESULT
    pickle_out = open("result_qsd2.pkl", "wb")
    pickle.dump(matrix_retrieval, pickle_out)
    pickle_out.close()
    
    return score

In [11]:
K = 10
query_folder = './qsd1_w3/'
textMatrixRetrieval = computeMatrixRetrievalText(bbdd_text,query_text,K=10)
score = evaluationTask1(textMatrixRetrieval,K,query_folder)
print(score)

IndexError: index 30 is out of bounds for axis 0 with size 30

# TASK 3

### Implement texture descriptors 
Descriptors:
* LBP
* DCT
* HoG

In [10]:
def zigzag(input):

    # initializing the variables
    # ----------------------------------
    h = 0
    v = 0
    vmin = 0
    hmin = 0
    vmax = input.shape[0]-1
    hmax = input.shape[1]-1

    i = 0
    output = np.zeros((vmax+1) * (hmax+1))
    # ----------------------------------

    while v <= vmax and h <= hmax:
        if np.mod(h + v, 2) == 0:
            if v == vmin:
                output[i] = input[v, h]     # if we got to the first line
                if h == hmax:
                    v += 1
                else:
                    h += 1
                i += 1
                continue
            if h == hmax and v < vmax:      # if we got to the last column
                output[i] = input[v, h]
                v += 1
                i += 1
            if v > vmin and h < hmax:       # all other cases
                output[i] = input[v, h]
                v -= 1
                h += 1
                i += 1
        else:
            if v == vmax and h <= hmax:     # if we got to the last line
                output[i] = input[v, h]
                h += 1
                i += 1
            if v != vmax and h == hmin:     # if we got to the first column
                output[i] = input[v, h]
                if v == vmax:
                    h += 1
                else:
                    v += 1
                i += 1
            if v < vmax and h > hmin:       # all other cases
                output[i] = input[v, h]
                v += 1
                h -= 1
                i += 1
        if v == vmax and h == hmax:         # bottom right element
            output[i] = input[v, h]
            break

    return output


In [11]:
def colorDescriptor(img,level,numberBins):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2Lab)
    
    #Calculem tamany del block
    height, width = img.shape[:2]
    block_height = int(height/level)
    block_width = int(width/level)
    hist_img = np.empty([0,0])
    colorx = ('x','y','z')
    canal = 0
    for i,col in enumerate(colorx): #Bucle per recorrer els canals de cada imatge  
        for r in range(level): #Bucle per recorrer els blocs d'una mateixa row
            for c in range(level): #Bucle per recorrer els blocs d'una mateixa column
                
                block = img[r*block_height:r*block_height+block_height, c*block_width:c*block_width+block_width] 
                hist = cv2.calcHist([block],[i],None,[numberBins],[0,256]) #Calculem histogrames per blocs
                hist_t = hist.transpose()      
                #print(hist_t.shape)
                #print(hist_img.shape)
                if (i==0) and (r==0) and (c==0):
                    hist_img = hist_t
                else:
                    hist_img = np.concatenate((hist_img, hist_t), axis = 1)

    cv2.normalize(hist_img, hist_img, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    return hist_img

In [12]:
def LbpDescriptor(img, level, numberBins, radius, n_poinys):
    #Calculem tamany del block
    height, width = img.shape[:2]
    block_height = int(height/level)
    block_width = int(width/level)
    hist_img = np.empty([0,0])
    colorx = ('x','y','z')
    canal = 0
    
    for r in range(level): #Bucle per recorrer els blocs d'una mateixa row
        for c in range(level): #Bucle per recorrer els blocs d'una mateixa column
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            block = gray[r*block_height:r*block_height+block_height, c*block_width:c*block_width+block_width]
            lbp = local_binary_pattern(block, n_points, radius, 'uniform')
            lbp = lbp.astype(np.uint8)
            hist = cv2.calcHist([lbp],[0],None,[numberBins],[0,256]) #Calculem histogrames per blocs
            hist_t = hist.transpose() 
            if (r==0) and (c==0):
                hist_img = hist_t
            else:
                hist_img = np.concatenate((hist_img, hist_t), axis = 1)
    cv2.normalize(hist_img, hist_img, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    return hist_img

In [13]:
def HogDescriptor(img):
    fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                        cells_per_block=(1, 1), visualize=True, multichannel=True)
    #print("1 - hog_image.shape", hog_image.shape)
    #hog_image = hog_image.astype(np.uint8)
    #print("2 - hog_image.shape", hog_image.shape)

    return hog_image

In [14]:
def DctDescriptor(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    I_dct = dct(dct(block.T).T)
    I_zz = zigzag(I_dct)
    I_dct = I_zz[0:nCoef]
    return I_dct

### Test query system using QSD1-W2 using only texture descriptors

#### 1. Define Parameters

In [15]:
# DEFINING GLOBAL VARIABLES
Level = 1
descriptor = 'hog'
radius = 5
n_points = 8 * radius

if descriptor == 'hog':
    numberBins = 500*500
    nCoef = numberBins*Level**2
    
if descriptor == 'LBP':
    numberBins = n_points + 1
    nCoef = numberBins*Level**2

if descriptor == 'DCT2D':
    numberBins = 32
    nCoef = numberBins*Level**2
    
if descriptor == 'color':
    numberBins = 256
    nCoef = numberBins*3*Level**2

#### 2. Compute BBDD descriptors

In [ ]:
folder = './bbdd/'
print(nCoef)
histogram_bbdd_matrix = np.empty([0, nCoef]) #Creem una matriu buida
print(histogram_bbdd_matrix.shape)

for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)
        if descriptor == 'LBP':
            hist_img = LbpDescriptor(img, Level, numberBins, radius, n_points)
        if descriptor == 'hog':
            resize_size = (500,500)
            image_shape = img.shape[:2]
            image = cv2.resize(img,resize_size)
            hist_img = HogDescriptor(image)
            hist_img = np.array(hist_img).flatten()
        if descriptor == 'DCT2D':
            hist_img = DctDescriptor(img)
        if descriptor == 'color':
            hist_img = colorDescriptor(img,Level,numberBins)
        print(hist_img.shape)
        histogram_bbdd_matrix = np.vstack((histogram_bbdd_matrix,hist_img))

#### 3. Compute QSD1_W3 denoised

In [ ]:
folder = './qsd1_w2/'
histogram_query_matrix = np.empty([0, nCoef]) #Creem una matriu buida

for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)
        if descriptor == 'LBP':
            hist_img = LbpDescriptor(img, Level, numberBins, radius, n_points)
        if descriptor == 'hog':
            resize_size = (500,500)
            image_shape = img.shape[:2]
            image = cv2.resize(img,resize_size)
            hist_img = HogDescriptor(image)
            hist_img = np.array(hist_img).flatten()
        if descriptor == 'DCT2D':
            hist_img = DctDescriptor(img)
        if descriptor == 'color':
            hist_img = colorDescriptor(img,Level,numberBins)
        print(hist_img.shape)
        histogram_query_matrix = np.vstack((histogram_query_matrix,hist_img))

#### 4. Evaluation using only texture descriptors

In [ ]:
# DEFINE EVALUATION FUNCTIONS

def computeMatrixRetrieval(histogram_bbdd, histogram_query, K=10):
    dst = np.zeros((30, 279))
    matrix_retrieval = np.zeros((30,K))

    for query_image_index in range(0,len(histogram_query)): 
        for bbdd_image_index in range(0, len(histogram_bbdd)):

            dst[query_image_index,bbdd_image_index] = distance.braycurtis(histogram_query[query_image_index,:], histogram_bbdd[bbdd_image_index,:])

        matrix_retrieval[query_image_index,:] = np.argsort(dst[query_image_index,:])[:K]

    #Convertir idx a list of lists
    matrix_retrieval = matrix_retrieval.astype(int)
    matrix_retrieval_lst = matrix_retrieval.tolist()
    return matrix_retrieval_lst


def evaluationTask1(matrix_retrieval, K, query_folder):  
    #LLEGIR GT
    with open(query_folder + 'gt_corresps.pkl', 'rb') as fd:
            ll = pickle.load(fd)
            #print(ll)
    gt = np.empty((0,0))

    ll_prp = np.zeros((len(ll),1))

    for i in range(0,len(ll)):
        ll_prp[i] = ll[i][0]

    ll_prp_lst = ll_prp.tolist()
    #print(ll_prp_lst)

    mapak = metrics.average_precision.mapk(ll_prp_lst, matrix_retrieval, k=K)    
    score = mapak*100
    #print("SCORE: ",score, "%")

    #GUARDAR PICKLE RESULT
    pickle_out = open("result_qsd2.pkl", "wb")
    pickle.dump(matrix_retrieval, pickle_out)
    pickle_out.close()
    
    return score

In [ ]:
K=3
folder = './qsd1_w2/'
matrix_retrieval = computeMatrixRetrieval(histogram_bbdd_matrix, histogram_query_matrix, K)
print(matrix_retrieval)
evaluationTask1(matrix_retrieval, K, folder)

# TASK4 - Combine descriptors


## text + color

In [ ]:
# DEFINING GLOBAL VARIABLES
Level = 1
numberBins= 256
nCoef_color = numberBins*3*Level**2

#### 1. BBDD

In [ ]:
import codecs
folder = './bbdd/'
histogram_bbdd_matrix_color = np.empty([0, nCoef_color]) #Creem una matriu buida
print(histogram_bbdd_matrix_color.shape)
textfolder = './bbdd_text/'
bbdd_text = []

for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)
        
        #COLOR
        hist_img_color = colorDescriptor(img,Level,numberBins)
        histogram_bbdd_matrix_color = np.vstack((histogram_bbdd_matrix_color,hist_img_color))



for filename in sorted(listdir(textfolder)):
    if(filename != '.DS_Store' and ('txt' in filename)):
        with codecs.open(textfolder + filename,'r',encoding='latin1') as f:
            if  os.stat(textfolder + filename).st_size == 0:
                line = ''
                bbdd_text.append(line)
                #print(filename + line)
            else:
                for line in f.readlines():
                    lim1 = line.find("'",1)
                    lim2 = line.find("'",2)
                    bbdd_text.append(line[lim1+1:lim2])

#### 2. QSD1_W3

In [ ]:
### TEXT
folder = './denoising_qsd1_w3/'
histogram_query_matrix_color = np.empty([0, nCoef_color]) #Creem una matriu buida
print(histogram_query_matrix_color.shape)
query_text = []
import pytesseract
with open('qsd1_w3_text.pkl', 'rb') as fd:
    ll = pickle.load(fd)
    #print(ll)

count = 0
for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)
        
        #COLOR
        hist_img_color = colorDescriptor(img,Level,numberBins)
        histogram_query_matrix_color = np.vstack((histogram_query_matrix_color,hist_img_color))

        #TEXT
        img = img[ll[count][1]:ll[count][3],ll[count][0]:ll[count][2]]
        extractedInformation = pytesseract.image_to_string(img)
        query_text.append(extractedInformation)
        count = count + 1

#### 3. Evaluation text + color

In [ ]:
def computeMatrixRetrieval_combText(histogram_bbdd, histogram_query, bbdd_text, query_text, K=10):
    dst_color = np.zeros((30, 279))
    dst_tex = np.zeros((30, 279))
    matrix_retrieval = np.zeros((30,K))

    for query_image_index_c in range(0,len(histogram_query)): 
        for bbdd_image_index_c in range(0, len(histogram_bbdd)):

            dst_color[query_image_index_c,bbdd_image_index_c] = distance.braycurtis(histogram_query[query_image_index_c,:], histogram_bbdd[bbdd_image_index_c,:])

    for query_text_index in range(0,len(query_text)):
        for bbdd_text_index in range(0,len(bbdd_text)-1):
            dst_tex[query_text_index,bbdd_text_index] = levenshtein(query_text[query_text_index], bbdd_text[bbdd_text_index])

            
    dst = ((5/6)*dst_tex) + ((1/6)*dst_color)
    for query_image_index in range(0,len(dst)):
        matrix_retrieval[query_image_index,:] = np.argsort(dst[query_image_index,:])[:K]

    #Convertir idx a list of lists
    matrix_retrieval = matrix_retrieval.astype(int)
    matrix_retrieval_lst = matrix_retrieval.tolist()
    return matrix_retrieval_lst



In [ ]:
K=10
folder = './qsd1_w3/'
matrix_retrieval = computeMatrixRetrieval_combText(histogram_bbdd_matrix_color, histogram_query_matrix_color, bbdd_text,query_text, K)
evaluationTask1(matrix_retrieval, K, folder)

## text + texture

#### 1. BBDD

In [ ]:
folder = './bbdd/'
nCoef_texture = 500*500
histogram_bbdd_matrix_texture = np.empty([0, nCoef_texture]) #Creem una matriu buida
print(histogram_bbdd_matrix_texture.shape)


for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)

        #TEXTURE
        resize_size = (500,500)
        image_shape = img.shape[:2]
        image = cv2.resize(img,resize_size)
        hist_img_texture = HogDescriptor(image)
        hist_img_texture = np.array(hist_img_texture).flatten()
        histogram_bbdd_matrix_texture = np.vstack((histogram_bbdd_matrix_texture,hist_img_texture))


# Read txt from bbdd_text
textfolder = './bbdd_text/'
bbdd_text = []
for filename in sorted(listdir(textfolder)):
    if(filename != '.DS_Store' and ('txt' in filename)):
        with codecs.open(textfolder + filename,'r',encoding='latin1') as f:
            if  os.stat(textfolder + filename).st_size == 0:
                line = ''
                bbdd_text.append(line)
                #print(filename + line)
            else:
                for line in f.readlines():
                    lim1 = line.find("'",1)
                    lim2 = line.find("'",2)
                    bbdd_text.append(line[lim1+1:lim2])
        


#### 2. QSD1_W3

In [ ]:
folder = './denoising_qsd1_w3/'
histogram_query_matrix_texture = np.empty([0, nCoef_texture]) #Creem una matriu buida
print(histogram_query_matrix_texture.shape)

query_text = []
import pytesseract
with open('qsd1_w3_text.pkl', 'rb') as fd:
    ll = pickle.load(fd)
    #print(ll)

count = 0
for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)

        #TEXTURE
        resize_size = (500,500)
        image_shape = img.shape[:2]
        image = cv2.resize(img,resize_size)
        hist_img_texture = HogDescriptor(image)
        hist_img_texture = np.array(hist_img_texture).flatten()
        histogram_query_matrix_texture = np.vstack((histogram_query_matrix_texture,hist_img_texture))
        
        
        #TEXT
        img = img[ll[count][1]:ll[count][3],ll[count][0]:ll[count][2]]
        extractedInformation = pytesseract.image_to_string(img)
        query_text.append(extractedInformation)
        count = count + 1

#### 3. Evaluation text + texture

In [ ]:
def computeMatrixRetrieval_combText(histogram_bbdd, histogram_query, bbdd_text, query_text, K=10):
    dst_color = np.zeros((30, 279))
    dst_tex = np.zeros((30, 279))
    matrix_retrieval = np.zeros((30,K))

    for query_image_index_c in range(0,len(histogram_query)): 
        for bbdd_image_index_c in range(0, len(histogram_bbdd)):

            dst_color[query_image_index_c,bbdd_image_index_c] = distance.braycurtis(histogram_query[query_image_index_c,:], histogram_bbdd[bbdd_image_index_c,:])

    for query_text_index in range(0,len(query_text)):
        for bbdd_text_index in range(0,len(bbdd_text)-1):
            dst_tex[query_text_index,bbdd_text_index] = levenshtein(query_text[query_text_index], bbdd_text[bbdd_text_index])

            
    dst = ((3/4)*dst_tex) + ((1/3)*dst_color)
    for query_image_index in range(0,len(dst)):
        matrix_retrieval[query_image_index,:] = np.argsort(dst[query_image_index,:])[:K]

    #Convertir idx a list of lists
    matrix_retrieval = matrix_retrieval.astype(int)
    matrix_retrieval_lst = matrix_retrieval.tolist()
    return matrix_retrieval_lst



In [ ]:
K=3
folder = './qsd1_w3/'
matrix_retrieval = computeMatrixRetrieval_combText(histogram_bbdd_matrix_texture, histogram_query_matrix_texture, bbdd_text,query_text, K)
evaluationTask1(matrix_retrieval, K, folder)

## texture + color

In [18]:
# DEFINING GLOBAL VARIABLES

nCoef_texture = 500*500
numberBins_color = 256
nCoef_color = numberBins_color*3*Level**2


#### 1. BBDD

In [ ]:
folder = './bbdd/'
histogram_bbdd_matrix_color = np.empty([0, nCoef_color]) #Creem una matriu buida
histogram_bbdd_matrix_texture = np.empty([0, nCoef_texture]) #Creem una matriu buida
print(histogram_bbdd_matrix_color.shape)
print(histogram_bbdd_matrix_texture.shape)


for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)
        
        #COLOR
        hist_img_color = colorDescriptor(img,Level,numberBins_color)
        histogram_bbdd_matrix_color = np.vstack((histogram_bbdd_matrix_color,hist_img_color))

        #TEXTURE
        resize_size = (500,500)
        image_shape = img.shape[:2]
        image = cv2.resize(img,resize_size)
        hist_img_texture = HogDescriptor(image)
        hist_img_texture = np.array(hist_img_texture).flatten()
        histogram_bbdd_matrix_texture = np.vstack((histogram_bbdd_matrix_texture,hist_img_texture))


#### 2. QSD1_W3

In [ ]:
folder = './denoising_qsd1_w3/'
histogram_query_matrix_color = np.empty([0, nCoef_color]) #Creem una matriu buida
histogram_query_matrix_texture = np.empty([0, nCoef_texture]) #Creem una matriu buida
print(histogram_query_matrix_color.shape)
print(histogram_query_matrix_texture.shape)


for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)
        
        #COLOR
        hist_img_color = colorDescriptor(img,Level,numberBins_color)
        histogram_query_matrix_color = np.vstack((histogram_query_matrix_color,hist_img_color))

        #TEXTURE
        resize_size = (500,500)
        image_shape = img.shape[:2]
        image = cv2.resize(img,resize_size)
        hist_img_texture = HogDescriptor(image)
        hist_img_texture = np.array(hist_img_texture).flatten()
        histogram_query_matrix_texture = np.vstack((histogram_query_matrix_texture,hist_img_texture))

#### 3. Evaluation texture + color

In [ ]:
def computeMatrixRetrieval_colorTexture(histogram_bbdd_color, histogram_query_color, histogram_bbdd_tex, histogram_query_tex , K=10):
    dst_color = np.zeros((30, 279))
    dst_tex = np.zeros((30, 279))
    matrix_retrieval = np.zeros((30,K))

    for query_image_index_c in range(0,len(histogram_query_color)): 
        for bbdd_image_index_c in range(0, len(histogram_bbdd_color)):

            dst_color[query_image_index_c,bbdd_image_index_c] = distance.braycurtis(histogram_query_color[query_image_index_c,:], histogram_bbdd_color[bbdd_image_index_c,:])

    for query_image_index_t in range(0,len(histogram_query_tex)): 
        for bbdd_image_index_t in range(0, len(histogram_bbdd_tex)):
            
            dst_tex[query_image_index_t,bbdd_image_index_t] = distance.braycurtis(histogram_query_tex[query_image_index_t,:], histogram_bbdd_tex[bbdd_image_index_t,:])

            
    dst = ((5/6)*dst_tex) + ((1/6)*dst_color)
    for query_image_index in range(0,len(dst)):
        matrix_retrieval[query_image_index,:] = np.argsort(dst[query_image_index,:])[:K]

    #Convertir idx a list of lists
    matrix_retrieval = matrix_retrieval.astype(int)
    matrix_retrieval_lst = matrix_retrieval.tolist()
    return matrix_retrieval_lst

In [ ]:
K=3
folder = './qsd1_w3/'
matrix_retrieval = computeMatrixRetrieval_colorTexture(histogram_bbdd_matrix_color, histogram_query_matrix_color, histogram_bbdd_matrix_texture,histogram_query_matrix_texture, K)
evaluationTask1(matrix_retrieval, K, folder)

## text + color + texture

#### 1. BBDD

In [27]:
# DEFINING GLOBAL VARIABLES
Level = 1
nCoef_texture = 500*500
numberBins_color = 256
nCoef_color = numberBins_color*3*Level**2

folder = './bbdd/'
histogram_bbdd_matrix_color = np.empty([0, nCoef_color]) #Creem una matriu buida
histogram_bbdd_matrix_texture = np.empty([0, nCoef_texture]) #Creem una matriu buida
print(histogram_bbdd_matrix_color.shape)
print(histogram_bbdd_matrix_texture.shape)


for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)
        
        #COLOR
        hist_img_color = colorDescriptor(img,Level,numberBins_color)
        histogram_bbdd_matrix_color = np.vstack((histogram_bbdd_matrix_color,hist_img_color))

        #TEXTURE
        resize_size = (500,500)
        image_shape = img.shape[:2]
        image = cv2.resize(img,resize_size)
        hist_img_texture = HogDescriptor(image)
        hist_img_texture = np.array(hist_img_texture).flatten()
        histogram_bbdd_matrix_texture = np.vstack((histogram_bbdd_matrix_texture,hist_img_texture))
        


# Read txt from bbdd_text
textfolder = './bbdd_text/'
bbdd_text = []
for filename in sorted(listdir(textfolder)):
    if(filename != '.DS_Store' and ('txt' in filename)):
        with codecs.open(textfolder + filename,'r',encoding='latin1') as f:
            if  os.stat(textfolder + filename).st_size == 0:
                line = ''
                bbdd_text.append(line)
                #print(filename + line)
            else:
                for line in f.readlines():
                    lim1 = line.find("'",1)
                    lim2 = line.find("'",2)
                    bbdd_text.append(line[lim1+1:lim2])
        



(0, 768)
(0, 250000)
 Processing image...   ./bbdd/bbdd_00278.jpg

#### 2. QSD1_W3

In [18]:
folder = './qst1_w3_denoise/'
histogram_query_matrix_color = np.empty([0, nCoef_color]) #Creem una matriu buida
histogram_query_matrix_texture = np.empty([0, nCoef_texture]) #Creem una matriu buida
print(histogram_query_matrix_color.shape)
print(histogram_query_matrix_texture.shape)
query_text = []
import pytesseract
with open('qst1_w3_text.pkl', 'rb') as fd:
    ll = pickle.load(fd)
    #print(ll)

count = 0

for filename in sorted(listdir(folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(folder+filename), end="")
        img = cv2.imread(folder + filename)
        
        #COLOR
        hist_img_color = colorDescriptor(img,Level,numberBins_color)
        histogram_query_matrix_color = np.vstack((histogram_query_matrix_color,hist_img_color))

        #TEXTURE
        resize_size = (500,500)
        image_shape = img.shape[:2]
        image = cv2.resize(img,resize_size)
        hist_img_texture = HogDescriptor(image)
        hist_img_texture = np.array(hist_img_texture).flatten()
        histogram_query_matrix_texture = np.vstack((histogram_query_matrix_texture,hist_img_texture))
        
        #TEXT
        img = img[ll[count][0][1]:ll[count][0][3],ll[count][0][0]:ll[count][0][2]]
        extractedInformation = pytesseract.image_to_string(img)
        query_text.append(extractedInformation)
        count = count + 1

(0, 768)
(0, 250000)
 Processing image...   ./qst1_w3_denoise/00049.jpg

#### 3. Evaluation text + color

In [24]:
def computeMatrixRetrieval_colorTexture(histogram_bbdd_color, histogram_query_color, histogram_bbdd_tex, histogram_query_tex, bbdd_text, query_text, K=10):
    dst_color = np.zeros((50, 279))
    dst_tex = np.zeros((50, 279))
    dst_text = np.zeros((50, 279))
    matrix_retrieval = np.zeros((50,K))

    for query_image_index_c in range(0,len(histogram_query_color)): 
        for bbdd_image_index_c in range(0, len(histogram_bbdd_color)):

            dst_color[query_image_index_c,bbdd_image_index_c] = distance.braycurtis(histogram_query_color[query_image_index_c,:], histogram_bbdd_color[bbdd_image_index_c,:])

    for query_image_index_t in range(0,len(histogram_query_tex)): 
        for bbdd_image_index_t in range(0, len(histogram_bbdd_tex)):
            
            dst_tex[query_image_index_t,bbdd_image_index_t] = distance.braycurtis(histogram_query_tex[query_image_index_t,:], histogram_bbdd_tex[bbdd_image_index_t,:])
    
    for query_text_index in range(0,len(query_text)):
        for bbdd_text_index in range(0,len(bbdd_text)-1):
            dst_text[query_text_index,bbdd_text_index] = levenshtein(query_text[query_text_index], bbdd_text[bbdd_text_index])       
            
    dst = ((1/3)*dst_tex) + ((1/3)*dst_color) + ((1/3)*dst_text)
    for query_image_index in range(0,len(dst)):
        matrix_retrieval[query_image_index,:] = np.argsort(dst[query_image_index,:])[:K]

    #Convertir idx a list of lists
    matrix_retrieval = matrix_retrieval.astype(int)
    matrix_retrieval_lst = matrix_retrieval.tolist()
    return matrix_retrieval_lst

In [24]:
matrix_retrieval = computeMatrixRetrieval_colorTexture(histogram_bbdd_matrix_color, histogram_query_matrix_color, histogram_bbdd_matrix_texture,histogram_query_matrix_texture, bbdd_text, query_text, K)

In [31]:
#Create pikel
matrix_retrieval_aux = []
for x in matrix_retrieval:
    matrix_retrieval_aux.append([x]) 

pickle_out = open('result_QST1_w3.pkl','wb')
pickle.dump(matrix_retrieval_aux,pickle_out)
pickle_out.close()

In [25]:
matrix_retrieval

[[208, 215, 223, 184, 92, 272, 10, 176, 131, 210],
 [215, 208, 223, 92, 184, 272, 176, 10, 131, 210],
 [61, 29, 9, 87, 234, 27, 270, 244, 248, 3],
 [217, 213, 172, 107, 152, 159, 98, 269, 260, 204],
 [99, 54, 105, 130, 235, 229, 134, 205, 14, 5],
 [110, 192, 219, 151, 203, 253, 265, 150, 218, 278],
 [118, 243, 205, 14, 82, 5, 66, 116, 6, 70],
 [79, 13, 196, 71, 230, 166, 236, 150, 195, 197],
 [54, 99, 105, 130, 235, 229, 134, 205, 14, 5],
 [50, 224, 197, 89, 84, 13, 79, 196, 163, 142],
 [166, 230, 244, 13, 79, 196, 23, 71, 142, 44],
 [89, 197, 50, 224, 84, 79, 13, 196, 245, 117],
 [57, 120, 42, 129, 143, 194, 15, 254, 249, 248],
 [195, 88, 24, 68, 65, 79, 196, 158, 13, 101],
 [51, 121, 128, 139, 8, 14, 205, 107, 3, 152],
 [165, 163, 82, 254, 143, 168, 101, 224, 81, 44],
 [163, 165, 82, 254, 143, 168, 101, 30, 89, 224],
 [165, 163, 82, 254, 143, 168, 101, 197, 81, 70],
 [163, 165, 82, 254, 143, 101, 81, 210, 224, 70],
 [230, 166, 244, 79, 196, 13, 23, 85, 219, 71],
 [201, 251, 55, 220, 

In [ ]:
K=10
folder = './qsd1_w3/'
matrix_retrieval = computeMatrixRetrieval_colorTexture(histogram_bbdd_matrix_color, histogram_query_matrix_color, histogram_bbdd_matrix_texture,histogram_query_matrix_texture, bbdd_text, query_text, K)
evaluationTask1(matrix_retrieval, K, folder)

# TASK 5

In [32]:
query2folder = './qst2_w3/'
# CREEM UN DIRECTORI ON GUARDEM LES IMATGES SENSE SOROLL
path = 'qst2_w3_denoise'
if not os.path.exists(path):
    os.mkdir(path)


for filename in sorted(listdir(query2folder)):
    if(filename != '.DS_Store' and ('jpg' in filename)):
        #print ("\r Processing image...   {}".format(query2folder+filename), end="")
        query_img = cv2.imread(query2folder + filename)
        gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
        noise = detectnoise(gray)
        if noise:
            denoise_img = cv2.bilateralFilter(query_img, 9, 120, 120)
            denoise_img_2 = cv2.fastNlMeansDenoisingColored(denoise_img,None,15,15,10,21) 
            #plt.figure()
            #plt.imshow(denoise_img)
            #plt.figure()
            #plt.imshow(denoise_img_2)
        else:
            denoise_img_2 = query_img
        
        cv2.imwrite(os.path.join(path,filename), denoise_img_2)



In [2]:
query2folder = './qst2_w3/'

In [4]:
def waterShed(query_img, color_space, sat_chan):
    """
    Applies watershed to detect foreground and baskground

    :param query_img: (ndarray) query image
    :param sat: (ndarray) query image saturation
    :param part: (int) one paint(0), two paints left part(1),  two paints right part(2)
    :return: mask: predicted mask
    """
    
    if sat_chan:
        ret, thresh = cv2.threshold(color_space, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        thresh = 255 - thresh
    else:
        thresh = k_means(color_space)

    # noise removal
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    # fillin holes
    kernel = np.ones((5, 5), np.uint8)
    opening = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=2)

    # sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)  # Fals. El sure

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 3)
    ret, sure_fg = cv2.threshold(dist_transform, 0.13 * dist_transform.max(), 255, 0)  # 0.15 26.57%

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    markers[markers != 0] = 200

    border_top = int(sat.shape[0] * .03)
    borde_lat = int(sat.shape[1] * .02)

    # define known sure background
    markers[0:border_top, :] = 100
    sure_fg[0:border_top, :] = 100

    markers[len(markers) - border_top:len(markers) - 1, :] = 100
    sure_fg[len(markers) - border_top:len(markers) - 1, :] = 100

    markers[:, 0:borde_lat] = 100
    sure_fg[:, 0:borde_lat] = 100
    markers[:, len(markers[0, :]) - borde_lat:len(markers[0, :]) - 1] = 100
    sure_fg[:, len(markers[0, :]) - borde_lat:len(markers[0, :]) - 1] = 100

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Now, mark the region of unknown with zero
    markers[sure_fg == 0] = 0

    markersres = cv2.watershed(query_img, markers)

    markersres[markersres == 201] = 255
    markersres[markersres == 101] = 0
    markersres[markersres == -1] = 255

    # Make the mask binary
    markersres = markersres / np.max(markersres)
            
    # Remove imperfections with morphological filers (noise)
    markersres = cv2.morphologyEx(markersres, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=1)

    # Set predicted mask
    mask_pred = markersres.astype('uint8')

    # Get connected components
    ret, labels = cv2.connectedComponents(mask_pred)

    # Initialize mask for paintings
    mask = np.zeros((mask_pred.shape[0], mask_pred.shape[1], 2), dtype="uint8")

    # If exist more than two connected components (background and one painting)
    if ret > 2:

        # Compute area of each connected component
        area = []
        for i, lab in enumerate(np.unique(labels)):
            area.append(mask_pred[labels == lab].size)

        # Sort indexes from length of areas
        idx = sorted(range(len(area)), key=lambda k: area[k])

        # Check if more than three connected components (background and two paintings)
        if ret > 3:
            for i, j in enumerate(idx):
                # Remove smallest areas
                if (len(area)-i) > 3:
                    mask_pred[labels == j] = 0

        # Remove indexes labels where its pixels are now zero
        idx = [n for _, n in enumerate(idx) if np.sum(mask_pred[labels == n]) != 0]

        # Positions of non-zero elements of each painting
        obj1 = np.where(labels == idx[0])
        obj2 = np.where(labels == idx[1])

        # Determine if there exist two paintings
        if (area[idx[0]] > .28*area[idx[1]]):

            # Check that one connected component is not inside the other
            if (((np.min(obj1[1][:]) - np.max(obj2[1][:])) < 0) and ((np.max(obj1[1][:]) - np.min(obj2[1][:])) > 0)) \
                    or (((np.min(obj2[1][:]) - np.max(obj1[1][:])) < 0) and
                        ((np.max(obj2[1][:]) - np.min(obj1[1][:])) > 0)):
                mask = mask_pred
            else:
                # Set one mask to each painting
                mask[labels == idx[0], 0] = 1
                mask[labels == idx[1], 1] = 1

        else:
            # Set mask for the painting
            mask = mask_pred

    else:
        # Set mask for the painting
        mask = mask_pred

    ## POLYGON - Generate a polygon from mask corners ##

    mask_f = np.zeros((mask_pred.shape[0], mask_pred.shape[1], 2), dtype="uint8")

    # For two paintings
    if len(mask.shape) > 2:
        # Get Corners
        corners0 = detect_corners(mask[:, :, 0])

        # Draw polygon
        poly0 = Image.new('RGB', (mask_pred.shape[1], mask_pred.shape[0]), (0))
        pdraw0 = ImageDraw.Draw(poly0)
        pdraw0.polygon(xy=corners0, fill=(1), outline=None)
        mask_pred_f = np.array(poly0)[:, :, 0]
        mask_f[:, :, 0] = mask_pred_f

        # Get Corners
        corners1 = detect_corners(mask[:, :, 1])

        # Draw polygon
        poly1 = Image.new('RGB', (mask_pred.shape[1], mask_pred.shape[0]), (0))
        pdraw1 = ImageDraw.Draw(poly1)
        pdraw1.polygon(xy=corners1, fill=(1), outline=None)
        mask_f[:, :, 1] = np.array(poly1)[:, :, 0]
        mask_pred_f = mask_pred_f + mask_f[:, :, 1]

    # Just one painting
    else:
        # Get Corners
        corners = detect_corners(mask)

        # Draw polygon
        poly = Image.new('RGB', (mask_pred.shape[1], mask_pred.shape[0]), (0))
        pdraw = ImageDraw.Draw(poly)
        pdraw.polygon(xy=corners, fill=(1), outline=None)
        mask_pred_f = np.array(poly)[:, :, 0]
        mask_f = mask_pred_f

    return mask_pred_f, mask_f


def k_means(color_space):
    
    Z = color_space.reshape((-1))
    # convert to np.float32
    Z = np.float32(Z)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret, label, center = cv2.kmeans(Z, 5, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((gray.shape))

    mask = np.zeros(color_space.shape[:2],np.uint8)

    for l in range(len(center)):
        lab_pos = np.where(res2==center[l])
        if any(lab_pos[0]==0) or any(lab_pos[1]==0) or any(lab_pos[0]==color_space.shape[0]-1) or any(lab_pos[1]==color_space.shape[1]-1):
            continue
        else:
            mask[lab_pos[0],lab_pos[1]] = 255
    
    if np.sum(mask) == 0:
        pos = np.where(res2==res2[int(color_space.shape[0]/2),int(color_space.shape[1]/2)])
        mask[pos[0],pos[1]] = 255
        
    return mask



#### 1. Remove background

In [7]:
query2folder = './qst2_w3_denoise/'

for filename in sorted(listdir(query2folder)):
    if (filename != '.DS_Store' and ('jpg' in filename)):
        print ("\r Processing image...   {}".format(query2folder+filename), end="")
        query_img = cv2.imread(query2folder + filename)
        hsv = cv2.cvtColor(query_img, cv2.COLOR_BGR2HSV)
        gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
        
        # Get saturation and grayscale from query image
        sat = cv2.cvtColor(query_img, cv2.COLOR_BGR2HSV)[:,:,1]
        gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
        
        if ndimage.variance(sat)<ndimage.variance(gray):
            color_space = gray
            sat_channel=False
            
        else:
            color_space = sat
            sat_channel=True
            
        # Predict mask
        mask_pred,mask = waterShed(query_img, color_space, sat_channel)
        
        # Save masks
        path = './masks'
        if not os.path.exists(path):
            os.mkdir(path)

        cv2.imwrite('masks/'+filename[:-4]+'.png',mask_pred*255)
        if len(mask.shape) > 2:
            cv2.imwrite('masks/mask0_'+filename[:-4]+'.png',mask[:,:,0]*255)
            cv2.imwrite('masks/mask1_'+filename[:-4]+'.png',mask[:,:,1]*255)
        else:
            cv2.imwrite('masks/mask0_'+filename[:-4]+'.png',mask[:,:,0]*255)

 Processing image...   ./qst2_w3_denoise/00029.jpg

In [8]:
def crop(query_im, mask_crop):
    # Coordinates of non-black pixels.
    coords = np.argwhere(mask_crop)

    # Bounding box of non-black pixels.
    x0, y0 = coords.min(axis=0)
    x1, y1 = coords.max(axis=0) + 1   # slices are exclusive at the top

    # Get the contents of the bounding box.
    cropped = query_im[x0:x1, y0:y1]
    mask_cropped = mask_crop[x0:x1, y0:y1]
    
    return cropped, mask_cropped, x0, y0

#### 2. Detect text bounding boxes for each painting

In [9]:
query2masks = './masks/'

bboxes = []

for filename in sorted(listdir(query2masks)):
    if (filename != '.DS_Store' and ('png' in filename) and ('mask' in filename)):
        print ("\r Processing image...   {}".format(query2masks+filename), end="")
        
        mask = cv2.imread(query2masks + filename)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

        img_num = filename[6:11]

        query_img = cv2.imread(query2folder + img_num + '.jpg')

        cropped, mask_cropped, X0, Y0 = crop(query_img,mask)

        bbox = find_box(cropped)

        if len(bboxes)>int(img_num):
            bboxes[int(img_num)].append(bbox)
        else:
            bboxes.append([bbox])

pickle_out = open("qst2_w3_text.pkl", "wb")
pickle.dump(bboxes, pickle_out)
pickle_out.close()

 Processing image...   ./masks/mask1_00029.png

#### 3. Predict correspondences

In [54]:
histogram_query_matrix_color = np.empty([0, nCoef_color]) #Creem una matriu buida
histogram_query_matrix_texture = np.empty([0, nCoef_texture]) #Creem una matriu buida
query_text = []

with open('qst2_w3_text.pkl', 'rb') as fd:
    ll = pickle.load(fd)

for filename in sorted(listdir(query2masks)):
    if (filename != '.DS_Store' and ('png' in filename) and ('mask' in filename)):
        print ("\r Processing image...   {}".format(query2masks+filename), end="")
        
        mask = cv2.imread(query2masks + filename)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        
        img_num = filename[6:11]
        num_paint = int(filename[4])

        query_img = cv2.imread(query2folder + img_num + '.jpg')

        cropped, mask_cropped, X0, Y0 = crop(query_img,mask)
        
        #COLOR
        hist_img_color = colorDescriptor(cropped,Level,numberBins_color)
        histogram_query_matrix_color = np.vstack((histogram_query_matrix_color,hist_img_color))

        #TEXTURE
        resize_size = (500,500)
        image_shape = cropped.shape[:2]
        image = cv2.resize(cropped,resize_size)
        hist_img_texture = HogDescriptor(image)
        hist_img_texture = np.array(hist_img_texture).flatten()
        histogram_query_matrix_texture = np.vstack((histogram_query_matrix_texture,hist_img_texture))
        
        #TEXT
        img = cropped[ll[int(img_num)][num_paint][1]:ll[int(img_num)][num_paint][3],ll[int(img_num)][num_paint][0]:ll[int(img_num)][num_paint][2]]
        if img.shape[0]<4:
            query_text.append('')
        else:
            extractedInformation = pytesseract.image_to_string(img)
            query_text.append(extractedInformation)
        
        with open(img_num+'.txt', 'a') as txtf:
            if num_paint==0:
                txtf.write(extractedInformation)
            else:
                txtf.write('\n'+extractedInformation)
            txtf.close()


 Processing image...   ./masks/mask1_00029.png

#### 4. Evaluation

In [55]:
K=10
matrix_retrieval = computeMatrixRetrieval_colorTexture(histogram_bbdd_matrix_color, histogram_query_matrix_color, histogram_bbdd_matrix_texture,histogram_query_matrix_texture, bbdd_text, query_text, K)

In [56]:
final_result = []

count = 0
for l in ll:
    for i in range(len(l)):
        if i == 0:
            final_result.append([matrix_retrieval[count]])
        else:
            final_result[len(final_result)-1].append(matrix_retrieval[count])
        count += 1

pickle_out = open('result_QST2_w3.pkl','wb')
pickle.dump(final_result,pickle_out)
pickle_out.close()

